In [34]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas.api.types import is_numeric_dtype, is_string_dtype, is_categorical_dtype
import seaborn as sns
import math
import numpy as np
from lightgbm import LGBMRegressor

In [42]:
def values(model, X):

    pd_dict = {}
    for feat in X.columns:

        _feat_df = X[[feat]].copy()
        _oth_df = X[[i for i in X.columns if i != feat]].copy()

        _feat_df["helper"] = 1
        _oth_df["helper"] = 1
        _pd_df = _feat_df.merge(_oth_df, how="left", on="helper").drop(
            columns={"helper"}
        )
        _feat_df = _feat_df.drop(columns=['helper'])
        _oth_df = _oth_df.drop(columns=['helper'])

        if is_numeric_dtype(_pd_df[feat]):
            if _pd_df[feat].nunique() > 10:
                _pd_df = _pd_df[
                        _pd_df[feat].between(
                            _feat_df[feat].quantile(0.05), _feat_df[feat].quantile(0.95)
                        )
                    ]

        _X = X.copy().drop_duplicates()
        _X["train_hash"] = pd.util.hash_pandas_object(_X, index=False)
        _pd_df["hash"] = pd.util.hash_pandas_object(_pd_df, index=False)
        _pd_df = _pd_df.merge(_X[["train_hash"]], how='left', left_on='hash', right_on='train_hash')
        _pd_df["orig_row"] = _pd_df["train_hash"].isnull()*1
        _pd_df = _pd_df.drop(columns=["train_hash","hash"])

        if is_numeric_dtype(_pd_df[feat]):
            # Train quantile regressors
            lgb_lo = LGBMRegressor(objective='quantile', alpha=0.1).fit(_oth_df, _feat_df)
            lgb_hi = LGBMRegressor(objective='quantile', alpha=0.9).fit(_oth_df, _feat_df)

            _pd_df["lo_pred"] = lgb_lo.predict(_pd_df[[i for i in X.columns if i != feat]])
            _pd_df["hi_pred"] = lgb_hi.predict(_pd_df[[i for i in X.columns if i != feat]])

            _pd_df = _pd_df.drop(_pd_df[(_pd_df["orig_row"]==0)&((_pd_df[feat] > _pd_df["hi_pred"]) | (_pd_df[feat] < _pd_df["lo_pred"]))].index)
            _pd_df = _pd_df.drop(columns=["orig_row","lo_pred","hi_pred"])
        
        _pd_df["pred"] = model.predict_proba(_pd_df)[:, 1]

        pd_dict[feat] = (
                _pd_df.groupby(feat)[["pred"]].mean().rename(columns={"pred": "PD"})
            )

    return pd_dict

In [43]:
from sklearn.datasets import fetch_openml

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
X["y"] = y
X = X.dropna(subset={"pclass", "age", "sibsp", "parch", "fare"})
y = X["y"]
X = X[["pclass", "age", "sibsp", "parch", "fare"]]

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X, y)

In [44]:
df = values(clf, X)